In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
file_list = drive.ListFile({'q': "'1iW1GMIWmDm6h6uGxJPjk2bt-7JcQJETM' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: Word_Rec.ipynb, id: 1wT6ydffIgr9vUadtYw35MMRhquVOS_vE
title: Train_X.csv, id: 1Ni7vZ53ssxLTP5-J8Zv3svn6L9W-YpEM
title: Train_Y.csv, id: 1hkIEneGvbb3QJ_UMPioNxPF1PT1MdZqr
title: OCR.ipynb, id: 1_DYFu440zgSozaS2W5i7LH018UQU-C1y
title: emnist-balanced-train.csv, id: 1HfeTC65KaW3FOE2httDR6wYySvdUbF4o
title: emnist-balanced-test.csv, id: 12dgVBkW6GCa-1dQkSMrLQ55ee-FUgv3M


In [0]:
train_x = drive.CreateFile({'id': '1Ni7vZ53ssxLTP5-J8Zv3svn6L9W-YpEM'})
train_x.GetContentFile('Train_X.csv')
train_y = drive.CreateFile({'id': '1hkIEneGvbb3QJ_UMPioNxPF1PT1MdZqr'})
train_y.GetContentFile('Train_Y.csv')

In [5]:
import numpy as np
import keras
import matplotlib.pyplot as plt
import pandas as pd
import os

Using TensorFlow backend.


In [6]:
data = pd.read_csv("Train_X.csv").values
X_train = data[:, 1:]
data = pd.read_csv("Train_Y.csv").values
Y = data[:, 1:]
X_train = np.reshape(X_train, (X_train.shape[0], 64, 64, 1)) / 255
#Y = np.reshape(Y, (Y.shape[0], Y.shape[1], 1))
print(X_train.shape)
print(Y.shape)

Y_train = np.zeros((Y.shape[0], Y.shape[1], 53))
for i in range(Y_train.shape[0]) :
  for j in range(Y_train[i].shape[0]) :
    arr = np.zeros((53))
    c = int(Y[i][j])
    arr[c] = 1
    Y_train[i][j] = arr
    
print(Y_train.shape)

(115320, 64, 64, 1)
(115320, 10)
(115320, 10, 53)


In [0]:
from keras.layers import Input, Conv2D, BatchNormalization, MaxPooling2D, Activation, LSTM, Flatten, Dense, Dropout, TimeDistributed, RepeatVector
def make_model(input_shape) :
  X = keras.models.Sequential()
  X.add(Conv2D(16, (3, 3), padding = 'same', input_shape = input_shape))
  X.add(BatchNormalization())
  X.add(Activation('relu'))
  X.add(MaxPooling2D((3, 3)))
  
  X.add(Conv2D(32, (3, 3), padding = 'same'))
  X.add(BatchNormalization())
  X.add(Activation('relu'))
  X.add(MaxPooling2D((3, 3)))
  
  X.add(Conv2D(64, (3, 3), padding = 'same'))
  X.add(BatchNormalization())
  X.add(Activation('relu'))
  X.add(MaxPooling2D((3, 3)))
  X.add(Flatten())
  #X.add(keras.layers.Reshape((16, 16)))
  #X_input = Input((16, 16))
  #time = TimeDistributed(X)(X_input)
  X.add(RepeatVector(10))
  X.add(LSTM(64, return_sequences = True))
  X.add(Dense(53, activation = 'softmax'))
  
 
  #model = keras.models.Model(inputs = X_input, outputs = Y1)
  
  X.summary()
  return X
  
  #return model

In [8]:
model = make_model((64, 64, 1))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 16)        160       
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 64, 16)        64        
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 16)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 21, 21, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 21, 21, 32)        4640      
_________________________________________________________________
batch_normalization_2 (Batch (None, 21, 21, 32)        128       
_________________________________________________________________
activation_2 (Activation)    (None, 21, 21, 32)        0         
__________

In [11]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(X_train, Y_train, batch_size = 512, epochs = 20, validation_split = 0.05)

Train on 109554 samples, validate on 5766 samples
Epoch 1/20
109554/109554 [==============================] - 48s 437us/step - loss: 0.6809 - acc: 0.7947 - val_loss: 0.7300 - val_acc: 0.7838
Epoch 2/20
109554/109554 [==============================] - 47s 425us/step - loss: 0.6785 - acc: 0.7953 - val_loss: 0.7292 - val_acc: 0.7848
Epoch 3/20
109554/109554 [==============================] - 46s 421us/step - loss: 0.6728 - acc: 0.7971 - val_loss: 0.7352 - val_acc: 0.7876
Epoch 4/20
109554/109554 [==============================] - 46s 422us/step - loss: 0.6712 - acc: 0.7975 - val_loss: 0.7383 - val_acc: 0.7806
Epoch 5/20
109554/109554 [==============================] - 46s 423us/step - loss: 0.6660 - acc: 0.7991 - val_loss: 0.7255 - val_acc: 0.7901
Epoch 6/20
109554/109554 [==============================] - 46s 421us/step - loss: 0.6630 - acc: 0.7999 - val_loss: 0.7255 - val_acc: 0.7899
Epoch 7/20
109554/109554 [==============================] - 46s 421us/step - loss: 0.6597 - acc: 0.8010 